In [3]:
#!/usr/bin/env python3
"""
EXPERIMENTAL GROUP - Anaerobic Digester Simulation (ADAPTIVE CONTROL)
======================================================================
Adaptive flow ratios + Emergency HRT correction when pH out of range.
Compare against BASELINE GROUP to evaluate control strategy.
"""

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION CONSTANTS
# =============================================================================

@dataclass
class ReactorConfig:
    """Reactor configuration parameters"""
    volume: float = 1.0  # L
    base_hrt: float = 24.0  # hours
    dose_interval: float = 1/60  # hours (1 minute)
    simulation_time: float = 48.0  # hours (2 days)
    ambient_temp: float = 25.0  # °C
    temp_fluctuation: float = 2.78  # °C (5°F converted)

@dataclass
class InputProperties:
    """Properties of each input stream based on literature values"""
    # Fermented Urine
    urine_pH: float = 9.1
    urine_EC: float = 22.5  # mS/cm (midpoint 20-25)
    urine_buffer: float = 490.0  # meq/L
    urine_Fe: float = 0.9  # mg/L (midpoint 0.3-1.5)
    urine_Zn: float = 0.4
    urine_Cu: float = 0.065
    urine_Mn: float = 0.06
    urine_Ni: float = 0.03

    # Red Wine
    wine_pH: float = 3.4
    wine_EC: float = 2.0
    wine_buffer: float = 80.0
    wine_Fe: float = 5.5
    wine_Zn: float = 2.0
    wine_Cu: float = 0.3
    wine_Mn: float = 1.0
    wine_Ni: float = 0.05

    # Molasses (diluted ~10%)
    molasses_pH: float = 5.5
    molasses_EC: float = 0.275
    molasses_buffer: float = 75.0
    molasses_Fe: float = 3.0
    molasses_Zn: float = 0.3
    molasses_Cu: float = 0.05
    molasses_Mn: float = 0.1
    molasses_Ni: float = 0.01

    # Spirulina Powder (10g/L suspension)
    spirulina_pH: float = 8.24
    spirulina_EC: float = 0.4
    spirulina_buffer: float = 10.0
    spirulina_Fe: float = 6.64
    spirulina_Zn: float = 0.04
    spirulina_Cu: float = 0.01
    spirulina_Mn: float = 0.01
    spirulina_Ni: float = 0.025

    # Iron Oxide (10g/L suspension)
    feox_pH: float = 7.0
    feox_EC: float = 0.01
    feox_buffer: float = 5.0
    feox_Fe: float = 7240.0
    feox_Zn: float = 0.0
    feox_Cu: float = 0.0
    feox_Mn: float = 0.0
    feox_Ni: float = 0.0

@dataclass
class TargetRanges:
    """Healthy bacteria operating ranges"""
    pH_min: float = 6.5
    pH_max: float = 7.5
    pH_target: float = 7.0
    EC_min: float = 0.2
    EC_max: float = 25.0
    EC_target: float = 10.0
    FeOx_min: float = 2.0
    FeOx_max: float = 4.0
    FeOx_target: float = 3.0
    Zn_min: float = 0.2
    Zn_max: float = 0.6
    Zn_target: float = 0.4
    Cu_min: float = 0.03
    Cu_max: float = 0.1
    Cu_target: float = 0.065
    Mn_min: float = 0.5
    Mn_max: float = 2.0
    Mn_target: float = 1.0
    Ni_min: float = 0.01
    Ni_max: float = 0.05
    Ni_target: float = 0.03

@dataclass
class ModelCoefficients:
    """ODE model coefficients"""
    EC_temp_coeff: float = 0.02
    k1_pH_EC: float = 0.5
    tanh_smoothing: float = 0.1
    pH_temp_coeff: float = 0.03
    k_decay: float = 0.0167
    pH_eq: float = 7.0
    pH_bio_decay: float = 0.4 / 24.0  # 0.4 pH drop per 24 hours from biological activity
    ORP_base: float = -220.0
    ORP_EC_coeff: float = 8.0
    ORP_pH_coeff: float = 40.0  # absolute value term
    ORP_Fe_coeff: float = 10.0
    ORP_TM_coeff: float = 8.0
    V_base: float = 0.3
    V_EC_coeff: float = 0.04
    V_pH_coeff: float = -0.025
    V_Fe_coeff: float = 0.02
    V_TM_coeff: float = 0.03
    sigma_ORP: float = 4.0
    sigma_V: float = 0.005

@dataclass
class ReactorState:
    """Current state of the reactor"""
    pH: float = 7.0
    EC: float = 5.0
    Zn: float = 0.4
    Cu: float = 0.065
    Mn: float = 1.0
    Ni: float = 0.03
    FeOx: float = 2.5
    temperature: float = 25.0
    time: float = 0.0
    citric_acid_added: float = 0.0
    lime_added: float = 0.0
    stressor_phase: str = "normal"

    def copy(self):
        return ReactorState(
            pH=self.pH, EC=self.EC, Zn=self.Zn, Cu=self.Cu, Mn=self.Mn, Ni=self.Ni,
            FeOx=self.FeOx, temperature=self.temperature, time=self.time,
            citric_acid_added=self.citric_acid_added,
            lime_added=self.lime_added, stressor_phase=self.stressor_phase
        )

    @property
    def TM_total(self) -> float:
        return self.Zn + self.Cu + self.Mn + self.Ni

@dataclass
class DoseRecord:
    """Record of a single dose"""
    time: float
    Q_urine: float
    Q_spirulina_feox: float
    Q_molasses_wine: float
    Q_total: float
    citric_acid: float
    lime: float
    pH: float
    EC: float
    Zn: float
    Cu: float
    Mn: float
    Ni: float
    FeOx: float
    temperature: float
    ORP: float
    voltage: float
    hrt: float
    phase: str

class AnaerobicDigesterSimulation:
    """Main simulation class - CONTROLLED with adaptive flow rates"""

    def __init__(self):
        self.config = ReactorConfig()
        self.inputs = InputProperties()
        self.targets = TargetRanges()
        self.coeffs = ModelCoefficients()
        self.state = ReactorState()
        self.history: List[DoseRecord] = []

        # Initialize with same starting conditions as BASELINE group
        self.state.pH = 7.0
        self.state.EC = 5.0
        self.state.Zn = 0.4
        self.state.Cu = 0.065
        self.state.Mn = 1.0
        self.state.Ni = 0.03
        self.state.FeOx = 3.0

    def calculate_temperature(self, time: float) -> float:
        """Calculate temperature with sinusoidal fluctuation"""
        base_fluctuation = self.config.temp_fluctuation * np.sin(2 * np.pi * time / 24)
        noise = np.random.normal(0, 0.5)
        return self.config.ambient_temp + base_fluctuation + noise

    def calculate_hrt(self, pH: float) -> float:
        """Calculate HRT based on pH - lower HRT when pH is out of range"""
        if pH < 6.5 or pH > 7.5:
            return 3.0  # Emergency: aggressive correction
        return self.config.base_hrt  # 24 hours

    def calculate_flow_rates(self, state: ReactorState, hrt: float) -> Tuple[float, float, float]:
        """Calculate flow rates using ODE-derived optimization with ADAPTIVE CONTROL"""
        total_flow_per_dose = (self.config.volume * 1000) / (hrt * 60)

        # Stream properties
        urine = {
            'pH': self.inputs.urine_pH, 'buffer': self.inputs.urine_buffer,
            'EC': self.inputs.urine_EC, 'Zn': self.inputs.urine_Zn,
            'Cu': self.inputs.urine_Cu, 'Mn': self.inputs.urine_Mn,
            'Ni': self.inputs.urine_Ni, 'FeOx': 0.0
        }

        spir_feox = {
            'pH': (self.inputs.spirulina_pH * self.inputs.spirulina_buffer +
                   self.inputs.feox_pH * self.inputs.feox_buffer) /
                  (self.inputs.spirulina_buffer + self.inputs.feox_buffer),
            'buffer': self.inputs.spirulina_buffer + self.inputs.feox_buffer,
            'EC': (self.inputs.spirulina_EC + self.inputs.feox_EC) / 2,
            'Zn': self.inputs.spirulina_Zn + self.inputs.feox_Zn,
            'Cu': self.inputs.spirulina_Cu + self.inputs.feox_Cu,
            'Mn': self.inputs.spirulina_Mn + self.inputs.feox_Mn,
            'Ni': self.inputs.spirulina_Ni + self.inputs.feox_Ni,
            'FeOx': 10.0
        }

        mol_wine = {
            'pH': (self.inputs.molasses_pH * self.inputs.molasses_buffer +
                   self.inputs.wine_pH * self.inputs.wine_buffer) /
                  (self.inputs.molasses_buffer + self.inputs.wine_buffer),
            'buffer': (self.inputs.molasses_buffer + self.inputs.wine_buffer) / 2,
            'EC': (self.inputs.molasses_EC + self.inputs.wine_EC) / 2,
            'Zn': (self.inputs.molasses_Zn + self.inputs.wine_Zn) / 2,
            'Cu': (self.inputs.molasses_Cu + self.inputs.wine_Cu) / 2,
            'Mn': (self.inputs.molasses_Mn + self.inputs.wine_Mn) / 2,
            'Ni': (self.inputs.molasses_Ni + self.inputs.wine_Ni) / 2,
            'FeOx': 0.0
        }

        targets = {
            'pH': self.targets.pH_target, 'EC': self.targets.EC_target,
            'Zn': self.targets.Zn_target, 'Cu': self.targets.Cu_target,
            'Mn': self.targets.Mn_target, 'Ni': self.targets.Ni_target,
            'FeOx': self.targets.FeOx_target
        }

        weights = {'pH': 100.0, 'FeOx': 10.0, 'Zn': 1.0, 'Cu': 1.0, 'Mn': 1.0, 'Ni': 1.0, 'EC': 0.1}

        best_cost = float('inf')
        best_fracs = (0.33, 0.34, 0.33)

        resolution = 20
        for i in range(resolution + 1):
            for j in range(resolution + 1 - i):
                f_u = max(0.05, i / resolution)
                f_sw = max(0.05, j / resolution)
                f_mw = max(0.05, 1.0 - f_u - f_sw)

                if f_mw < 0.05:
                    continue

                total = f_u + f_sw + f_mw
                f_u, f_sw, f_mw = f_u/total, f_sw/total, f_mw/total

                total_buffer = f_u * urine['buffer'] + f_sw * spir_feox['buffer'] + f_mw * mol_wine['buffer']
                pH_mix = (f_u * urine['buffer'] * urine['pH'] +
                         f_sw * spir_feox['buffer'] * spir_feox['pH'] +
                         f_mw * mol_wine['buffer'] * mol_wine['pH']) / total_buffer

                EC_mix = f_u * urine['EC'] + f_sw * spir_feox['EC'] + f_mw * mol_wine['EC']
                Zn_mix = f_u * urine['Zn'] + f_sw * spir_feox['Zn'] + f_mw * mol_wine['Zn']
                Cu_mix = f_u * urine['Cu'] + f_sw * spir_feox['Cu'] + f_mw * mol_wine['Cu']
                Mn_mix = f_u * urine['Mn'] + f_sw * spir_feox['Mn'] + f_mw * mol_wine['Mn']
                Ni_mix = f_u * urine['Ni'] + f_sw * spir_feox['Ni'] + f_mw * mol_wine['Ni']
                FeOx_mix = f_u * urine['FeOx'] + f_sw * spir_feox['FeOx'] + f_mw * mol_wine['FeOx']

                cost = 0.0
                cost += weights['pH'] * ((pH_mix - targets['pH']) / 2.0) ** 2
                cost += weights['EC'] * ((EC_mix - targets['EC']) / 10.0) ** 2
                cost += weights['Zn'] * ((Zn_mix - targets['Zn']) / 0.4) ** 2
                cost += weights['Cu'] * ((Cu_mix - targets['Cu']) / 0.065) ** 2
                cost += weights['Mn'] * ((Mn_mix - targets['Mn']) / 1.0) ** 2
                cost += weights['Ni'] * ((Ni_mix - targets['Ni']) / 0.03) ** 2
                cost += weights['FeOx'] * ((FeOx_mix - targets['FeOx']) / 3.0) ** 2

                if cost < best_cost:
                    best_cost = cost
                    best_fracs = (f_u, f_sw, f_mw)

        f_u, f_sw, f_mw = best_fracs

        # ADAPTIVE CORRECTION based on current state error
        pH_error = state.pH - targets['pH']
        if abs(pH_error) > 0.3:
            if pH_error > 0:  # Too alkaline
                f_mw = min(0.8, f_mw + 0.3)
                f_u = max(0.05, f_u - 0.2)
            else:  # Too acidic
                f_u = min(0.8, f_u + 0.3)
                f_mw = max(0.05, f_mw - 0.2)

        FeOx_error = state.FeOx - targets['FeOx']
        if FeOx_error < -0.5:
            f_sw = min(0.5, f_sw + 0.15)
        elif FeOx_error > 0.5:
            f_sw = max(0.1, f_sw - 0.1)

        total = f_u + f_sw + f_mw
        f_u, f_sw, f_mw = f_u/total, f_sw/total, f_mw/total

        return total_flow_per_dose * f_u, total_flow_per_dose * f_sw, total_flow_per_dose * f_mw

    def calculate_stressor_dose(self, time: float) -> Tuple[float, float, str]:
        """NO STRESS - removed pH shock for comparison"""
        return 0.0, 0.0, "normal"

    def update_pH(self, state: ReactorState, Q_u: float, Q_sw: float, Q_mw: float,
                  citric: float, lime: float, dt: float) -> float:
        Q_total = Q_u + Q_sw + Q_mw
        if Q_total == 0:
            Q_total = 0.001

        spir_feox_pH = (self.inputs.spirulina_pH * self.inputs.spirulina_buffer +
                       self.inputs.feox_pH * self.inputs.feox_buffer) / (
                       self.inputs.spirulina_buffer + self.inputs.feox_buffer)
        spir_feox_buffer = self.inputs.spirulina_buffer + self.inputs.feox_buffer

        mol_wine_pH = (self.inputs.molasses_pH * self.inputs.molasses_buffer +
                      self.inputs.wine_pH * self.inputs.wine_buffer) / (
                      self.inputs.molasses_buffer + self.inputs.wine_buffer)
        mol_wine_buffer = (self.inputs.molasses_buffer + self.inputs.wine_buffer) / 2

        total_buffer_flow = (Q_u * self.inputs.urine_buffer +
                            Q_sw * spir_feox_buffer +
                            Q_mw * mol_wine_buffer)

        if total_buffer_flow > 0:
            pH_mix = (Q_u * self.inputs.urine_buffer * self.inputs.urine_pH +
                     Q_sw * spir_feox_buffer * spir_feox_pH +
                     Q_mw * mol_wine_buffer * mol_wine_pH) / total_buffer_flow
        else:
            pH_mix = state.pH

        reactor_buffer = 100.0
        pH_citric_effect = -citric / (self.config.volume * reactor_buffer) * 2
        pH_lime_effect = lime / (self.config.volume * reactor_buffer) * 2

        Q_total_L_hr = Q_total / 1000 * 60

        dilution_term = (1/self.config.volume) * Q_total_L_hr * (pH_mix - state.pH)
        temp_term = -self.coeffs.pH_temp_coeff * (state.temperature - 25)
        decay_term = -self.coeffs.k_decay * (state.pH - self.coeffs.pH_eq)
        bio_decay_term = -self.coeffs.pH_bio_decay  # Constant acid production by bacteria

        dpH_dt = dilution_term + temp_term + decay_term + bio_decay_term
        new_pH = state.pH + dpH_dt * dt + pH_citric_effect + pH_lime_effect

        return np.clip(new_pH, 3.0, 11.0)

    def update_EC(self, state: ReactorState, Q_u: float, Q_sw: float, Q_mw: float, dt: float) -> float:
        Q_total = Q_u + Q_sw + Q_mw
        if Q_total == 0:
            Q_total = 0.001

        spir_feox_EC = (self.inputs.spirulina_EC + self.inputs.feox_EC) / 2
        mol_wine_EC = (self.inputs.molasses_EC + self.inputs.wine_EC) / 2

        EC_mix = (Q_u * self.inputs.urine_EC +
                 Q_sw * spir_feox_EC +
                 Q_mw * mol_wine_EC) / Q_total

        Q_total_L_hr = Q_total / 1000 * 60

        dilution_term = (1/self.config.volume) * Q_total_L_hr * (EC_mix - state.EC)
        temp_term = self.coeffs.EC_temp_coeff * (state.temperature - 25) * state.EC

        tanh_term = 1 - 0.5 * (1 + np.tanh((state.pH - 6) / self.coeffs.tanh_smoothing))
        pH_coupling_term = self.coeffs.k1_pH_EC * (state.pH - 6) * tanh_term

        dEC_dt = dilution_term + temp_term + pH_coupling_term

        new_EC = state.EC + dEC_dt * dt
        return np.clip(new_EC, 0.01, 50.0)

    def update_trace_metals(self, state: ReactorState, Q_u: float, Q_sw: float,
                           Q_mw: float, dt: float) -> Tuple[float, float, float, float]:
        Q_total = Q_u + Q_sw + Q_mw
        if Q_total == 0:
            return state.Zn, state.Cu, state.Mn, state.Ni

        Q_u_L_hr = Q_u / 1000 * 60
        Q_sw_L_hr = Q_sw / 1000 * 60
        Q_mw_L_hr = Q_mw / 1000 * 60

        spir_feox_Zn = self.inputs.spirulina_Zn + self.inputs.feox_Zn
        spir_feox_Cu = self.inputs.spirulina_Cu + self.inputs.feox_Cu
        spir_feox_Mn = self.inputs.spirulina_Mn + self.inputs.feox_Mn
        spir_feox_Ni = self.inputs.spirulina_Ni + self.inputs.feox_Ni

        mol_wine_Zn = (self.inputs.molasses_Zn + self.inputs.wine_Zn) / 2
        mol_wine_Cu = (self.inputs.molasses_Cu + self.inputs.wine_Cu) / 2
        mol_wine_Mn = (self.inputs.molasses_Mn + self.inputs.wine_Mn) / 2
        mol_wine_Ni = (self.inputs.molasses_Ni + self.inputs.wine_Ni) / 2

        dZn_dt = ((Q_u_L_hr / self.config.volume) * (self.inputs.urine_Zn - state.Zn) +
                  (Q_sw_L_hr / self.config.volume) * (spir_feox_Zn - state.Zn) +
                  (Q_mw_L_hr / self.config.volume) * (mol_wine_Zn - state.Zn))

        dCu_dt = ((Q_u_L_hr / self.config.volume) * (self.inputs.urine_Cu - state.Cu) +
                  (Q_sw_L_hr / self.config.volume) * (spir_feox_Cu - state.Cu) +
                  (Q_mw_L_hr / self.config.volume) * (mol_wine_Cu - state.Cu))

        dMn_dt = ((Q_u_L_hr / self.config.volume) * (self.inputs.urine_Mn - state.Mn) +
                  (Q_sw_L_hr / self.config.volume) * (spir_feox_Mn - state.Mn) +
                  (Q_mw_L_hr / self.config.volume) * (mol_wine_Mn - state.Mn))

        dNi_dt = ((Q_u_L_hr / self.config.volume) * (self.inputs.urine_Ni - state.Ni) +
                  (Q_sw_L_hr / self.config.volume) * (spir_feox_Ni - state.Ni) +
                  (Q_mw_L_hr / self.config.volume) * (mol_wine_Ni - state.Ni))

        new_Zn = np.clip(state.Zn + dZn_dt * dt, 0.0, 50.0)
        new_Cu = np.clip(state.Cu + dCu_dt * dt, 0.0, 50.0)
        new_Mn = np.clip(state.Mn + dMn_dt * dt, 0.0, 50.0)
        new_Ni = np.clip(state.Ni + dNi_dt * dt, 0.0, 50.0)

        return new_Zn, new_Cu, new_Mn, new_Ni

    def update_iron_oxide(self, state: ReactorState, Q_u: float, Q_sw: float,
                         Q_mw: float, dt: float) -> float:
        Q_total = Q_u + Q_sw + Q_mw
        if Q_total == 0:
            return state.FeOx

        urine_FeOx = self.inputs.urine_Fe / 1000
        spir_feox_FeOx = 10.0
        mol_wine_FeOx = (self.inputs.molasses_Fe + self.inputs.wine_Fe) / 2 / 1000

        Q_u_L_hr = Q_u / 1000 * 60
        Q_sw_L_hr = Q_sw / 1000 * 60
        Q_mw_L_hr = Q_mw / 1000 * 60

        dFeOx_dt = ((Q_u_L_hr / self.config.volume) * (urine_FeOx - state.FeOx) +
                   (Q_sw_L_hr / self.config.volume) * (spir_feox_FeOx - state.FeOx) +
                   (Q_mw_L_hr / self.config.volume) * (mol_wine_FeOx - state.FeOx))

        new_FeOx = state.FeOx + dFeOx_dt * dt
        return np.clip(new_FeOx, 0.0, 20.0)

    def calculate_outputs(self, state: ReactorState) -> Tuple[float, float]:
        Fe_normalized = state.FeOx
        TM_normalized = state.TM_total

        ORP = (self.coeffs.ORP_base -
               self.coeffs.ORP_EC_coeff * state.EC -
               self.coeffs.ORP_pH_coeff * abs(state.pH - 7) -
               self.coeffs.ORP_Fe_coeff * Fe_normalized -
               self.coeffs.ORP_TM_coeff * TM_normalized)

        ORP += np.random.normal(0, self.coeffs.sigma_ORP)

        EC_safe = max(0.1, state.EC)
        voltage = (self.coeffs.V_base +
                  self.coeffs.V_EC_coeff * np.log(EC_safe) +
                  self.coeffs.V_pH_coeff * (state.pH - 7) +
                  self.coeffs.V_Fe_coeff * Fe_normalized +
                  self.coeffs.V_TM_coeff * TM_normalized)

        voltage += np.random.normal(0, self.coeffs.sigma_V)

        return ORP, voltage

    def run_simulation(self):
        """Run the complete simulation with Arduino-style output"""
        dt = self.config.dose_interval
        n_steps = int(self.config.simulation_time / dt)

        print("=" * 160)
        print("ANAEROBIC DIGESTER SIMULATION - EXPERIMENTAL GROUP (ADAPTIVE CONTROL)")
        print("With 0.4 pH/day biological decay - 48 Hour Simulation")
        print("Adaptive flow ratios + Emergency HRT correction when pH out of range")
        print("=" * 160)
        print(f"{'Time':>8} | {'Phase':^15} | {'Q_urine':>8} | {'Q_sp+fe':>8} | {'Q_mw':>8} | "
              f"{'pH':>6} | {'EC':>6} | {'Zn':>6} | {'Cu':>6} | {'Mn':>6} | {'Ni':>6} | "
              f"{'FeOx':>6} | {'Temp':>5} | {'ORP':>8} | {'V_mfc':>6} | {'HRT':>5}")
        print(f"{'(hr)':>8} | {'':^15} | {'(mL)':>8} | {'(mL)':>8} | {'(mL)':>8} | "
              f"{'':>6} | {'mS/cm':>6} | {'mg/L':>6} | {'mg/L':>6} | {'mg/L':>6} | {'mg/L':>6} | "
              f"{'g/L':>6} | {'°C':>5} | {'(mV)':>8} | {'(V)':>6} | {'(hr)':>5}")
        print("-" * 160)

        for step in range(n_steps):
            current_time = step * dt

            self.state.temperature = self.calculate_temperature(current_time)
            citric, lime, phase = self.calculate_stressor_dose(current_time)

            if self.state.pH < 6.5:
                phase = "pH_LOW_CORR"
            elif self.state.pH > 7.5:
                phase = "pH_HIGH_CORR"

            self.state.stressor_phase = phase
            hrt = self.calculate_hrt(self.state.pH)
            Q_u, Q_sw, Q_mw = self.calculate_flow_rates(self.state, hrt)

            new_pH = self.update_pH(self.state, Q_u, Q_sw, Q_mw, citric, lime, dt)
            new_EC = self.update_EC(self.state, Q_u, Q_sw, Q_mw, dt)
            new_Zn, new_Cu, new_Mn, new_Ni = self.update_trace_metals(self.state, Q_u, Q_sw, Q_mw, dt)
            new_FeOx = self.update_iron_oxide(self.state, Q_u, Q_sw, Q_mw, dt)

            self.state.pH = new_pH
            self.state.EC = new_EC
            self.state.Zn = new_Zn
            self.state.Cu = new_Cu
            self.state.Mn = new_Mn
            self.state.Ni = new_Ni
            self.state.FeOx = new_FeOx
            self.state.time = current_time
            self.state.citric_acid_added += citric
            self.state.lime_added += lime

            ORP, voltage = self.calculate_outputs(self.state)

            record = DoseRecord(
                time=current_time, Q_urine=Q_u, Q_spirulina_feox=Q_sw,
                Q_molasses_wine=Q_mw, Q_total=Q_u + Q_sw + Q_mw,
                citric_acid=citric, lime=lime, pH=self.state.pH, EC=self.state.EC,
                Zn=self.state.Zn, Cu=self.state.Cu, Mn=self.state.Mn, Ni=self.state.Ni,
                FeOx=self.state.FeOx, temperature=self.state.temperature,
                ORP=ORP, voltage=voltage, hrt=hrt, phase=phase
            )
            self.history.append(record)

            # Print Arduino-style output every minute
            print(f"{current_time:8.2f} | {phase:^15} | {Q_u:8.3f} | {Q_sw:8.3f} | {Q_mw:8.3f} | "
                  f"{self.state.pH:6.2f} | {self.state.EC:6.2f} | {self.state.Zn:6.3f} | {self.state.Cu:6.3f} | "
                  f"{self.state.Mn:6.3f} | {self.state.Ni:6.3f} | {self.state.FeOx:6.2f} | {self.state.temperature:5.1f} | "
                  f"{ORP:8.1f} | {voltage:6.3f} | {hrt:5.1f}")

        self.print_summary()
        return self.history

    def print_summary(self):
        """Print final summary"""
        print("\n" + "=" * 80)
        print("FINAL STATE SUMMARY - EXPERIMENTAL GROUP (ADAPTIVE CONTROL)")
        print("=" * 80)

        final = self.history[-1]

        print(f"\n{'State':<20} | {'Final Value':>15} | {'Target Range':>20} | {'Status':>10}")
        print("-" * 80)

        pH_status = "OK" if self.targets.pH_min <= final.pH <= self.targets.pH_max else "OUT OF RANGE"
        print(f"{'pH':<20} | {final.pH:>15.2f} | {f'{self.targets.pH_min:.1f} - {self.targets.pH_max:.1f}':>20} | {pH_status:>10}")

        EC_status = "OK" if self.targets.EC_min <= final.EC <= self.targets.EC_max else "OUT OF RANGE"
        print(f"{'EC (mS/cm)':<20} | {final.EC:>15.2f} | {f'{self.targets.EC_min:.1f} - {self.targets.EC_max:.1f}':>20} | {EC_status:>10}")

        Zn_status = "OK" if self.targets.Zn_min <= final.Zn <= self.targets.Zn_max else "OUT OF RANGE"
        print(f"{'Zinc (mg/L)':<20} | {final.Zn:>15.3f} | {f'{self.targets.Zn_min:.2f} - {self.targets.Zn_max:.2f}':>20} | {Zn_status:>10}")

        Cu_status = "OK" if self.targets.Cu_min <= final.Cu <= self.targets.Cu_max else "OUT OF RANGE"
        print(f"{'Copper (mg/L)':<20} | {final.Cu:>15.3f} | {f'{self.targets.Cu_min:.2f} - {self.targets.Cu_max:.2f}':>20} | {Cu_status:>10}")

        Mn_status = "OK" if self.targets.Mn_min <= final.Mn <= self.targets.Mn_max else "OUT OF RANGE"
        print(f"{'Manganese (mg/L)':<20} | {final.Mn:>15.3f} | {f'{self.targets.Mn_min:.2f} - {self.targets.Mn_max:.2f}':>20} | {Mn_status:>10}")

        Ni_status = "OK" if self.targets.Ni_min <= final.Ni <= self.targets.Ni_max else "OUT OF RANGE"
        print(f"{'Nickel (mg/L)':<20} | {final.Ni:>15.3f} | {f'{self.targets.Ni_min:.2f} - {self.targets.Ni_max:.2f}':>20} | {Ni_status:>10}")

        FeOx_status = "OK" if self.targets.FeOx_min <= final.FeOx <= self.targets.FeOx_max else "OUT OF RANGE"
        print(f"{'Iron Oxide (g/L)':<20} | {final.FeOx:>15.2f} | {f'{self.targets.FeOx_min:.1f} - {self.targets.FeOx_max:.1f}':>20} | {FeOx_status:>10}")

        print("-" * 80)
        print(f"\n{'Output':<20} | {'Final Value':>15} | {'Baseline':>20}")
        print("-" * 80)
        print(f"{'ORP (mV)':<20} | {final.ORP:>15.1f} | {self.coeffs.ORP_base:>20.1f}")
        print(f"{'MFC Voltage (V)':<20} | {final.voltage:>15.3f} | {self.coeffs.V_base:>20.3f}")

        # Calculate total fluid usage
        total_urine = sum(r.Q_urine for r in self.history)
        total_spir_feox = sum(r.Q_spirulina_feox for r in self.history)
        total_mol_wine = sum(r.Q_molasses_wine for r in self.history)
        total_all = total_urine + total_spir_feox + total_mol_wine

        print("\n" + "=" * 80)
        print("TOTAL FLUID USAGE")
        print("=" * 80)
        print(f"\n{'Input Stream':<35} | {'Volume (mL)':>15} | {'Volume (L)':>12} | {'% of Total':>10}")
        print("-" * 80)
        print(f"{'Fermented Urine':<35} | {total_urine:>15.1f} | {total_urine/1000:>12.3f} | {total_urine/total_all*100:>9.1f}%")
        print(f"{'Spirulina + Iron Oxide':<35} | {total_spir_feox:>15.1f} | {total_spir_feox/1000:>12.3f} | {total_spir_feox/total_all*100:>9.1f}%")
        print(f"{'Molasses + Wine':<35} | {total_mol_wine:>15.1f} | {total_mol_wine/1000:>12.3f} | {total_mol_wine/total_all*100:>9.1f}%")
        print("-" * 80)
        print(f"{'TOTAL':<35} | {total_all:>15.1f} | {total_all/1000:>12.3f} | {'100.0':>9}%")
        print("=" * 80)

def main():
    np.random.seed(42)
    sim = AnaerobicDigesterSimulation()
    history = sim.run_simulation()
    return sim, history

if __name__ == "__main__":
    main()

ANAEROBIC DIGESTER SIMULATION - EXPERIMENTAL GROUP (ADAPTIVE CONTROL)
With 0.4 pH/day biological decay - 48 Hour Simulation
Adaptive flow ratios + Emergency HRT correction when pH out of range
    Time |      Phase      |  Q_urine |  Q_sp+fe |     Q_mw |     pH |     EC |     Zn |     Cu |     Mn |     Ni |   FeOx |  Temp |      ORP |  V_mfc |   HRT
    (hr) |                 |     (mL) |     (mL) |     (mL) |        |  mS/cm |   mg/L |   mg/L |   mg/L |   mg/L |    g/L |    °C |     (mV) |    (V) |  (hr)
----------------------------------------------------------------------------------------------------------------------------------------------------------------
    0.00 |     normal      |    0.069 |    0.243 |    0.382 |   7.00 |   5.00 |  0.400 |  0.065 |  1.000 |  0.030 |   3.00 |  25.2 |   -302.5 |  0.472 |  24.0
    0.02 |     normal      |    0.069 |    0.243 |    0.382 |   7.00 |   5.00 |  0.400 |  0.065 |  0.999 |  0.030 |   3.00 |  25.8 |   -302.9 |  0.468 |  24.0
    0.03 |